<a href="https://colab.research.google.com/github/DaniPopov/deepLearningAssignments-22961/blob/main/1stAssignmentDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMN 11

Student info: Daniel Popov, ID: 319038519



### Imports

In [1]:
import torch
from torch import Tensor

### סעיף א

In [2]:
def new_expand_as(A: Tensor, B: Tensor) -> Tensor:
    """
    Expands tensor A to the shape of tensor B by prepending singleton dimensions and replicating elements as necessary,
    serving as a custom version of PyTorch's expand_as function.

   Args:
        A (Tensor): The source tensor to be expanded.
        B (Tensor): The target tensor whose shape A should match after expansion.

    Returns:
        Tensor: A new tensor with the same data as A but expanded to the shape of B.

    Raises:
        ValueError: If A has more dimensions than B, or if A and B are not compatible for broadcasting.

    Example:
        >>> A = torch.arange(27).reshape(3,3,3)
        >>> B = torch.arange(9).reshape(3,3)
        >>> print(new_expand_as(B, A))
        tensor([[[0, 1, 2],
                 [3, 4, 5],
                 [6, 7, 8]],

                 [[0, 1, 2],
                 [3, 4, 5],
                 [6, 7, 8]],

                 [[0, 1, 2],
                 [3, 4, 5],
                 [6, 7, 8]]])
    """
    if A.dim() > B.dim():
        raise RuntimeError("expand_as: The source tensor cannot have more dimensions than the target tensor.")

    # Ensure A has the same number of dimensions as B by prepending ones
    while A.dim() < B.dim():
          A  = A.unsqueeze(0)

    # Reverse the shapes to start checking from the trailing dimensions
    A_shape_reversed = A.shape[::-1]
    B_shape_reversed = B.shape[::-1]

    # Check for broadcast compatibility
    for i in range(len(A_shape_reversed)):
        if A_shape_reversed[i] != 1 and A_shape_reversed[i] != B_shape_reversed[i]:
            raise RuntimeError(f"expand_as: The size of the source tensor ({A.shape}) must match the size of the target tensor ({ B.shape}) at non-singleton dimension.")

    C = A.clone()
    C_shape_reversed = C.shape[::-1]

    for i in range(len(C_shape_reversed)):
        if C.shape[i] ==  1 and C.shape[i] != B.shape[i]:
          C = torch.cat([C] * B.shape[i], dim=i)

    return  C

# Usage
A = torch.arange(27).reshape(3,3,3)
B = torch.arange(9).reshape(3,3)
print(new_expand_as(B, A))


tensor([[[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]],

        [[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]],

        [[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]]])


### סעיף ב

In [3]:
def broad_together(A: Tensor, B: Tensor) -> (bool, tuple):
    """
    Checks if two tensors, A and B, can be broadcast together according to broadcasting rules.
    It returns a tuple containing a boolean indicating whether the tensors are compatible
    for broadcasting and, if true, the resulting shape after broadcasting.

    Args:
        A (Tensor): The first tensor.
        B (Tensor): The second tensor.

    Returns:
        (bool, tuple): A tuple where the first element is a boolean indicating whether the tensors
        can be broadcast together, and the second element is the resulting broadcast shape if they can,
        otherwise None.

    Example:
        >>> A = torch.rand(3, 3, 3)
        >>> B = torch.arange(9).reshape(3, 3)
        >>> can_broadcast, broadcast_shape = broad_together(B, A)
        >>> print(can_broadcast, broadcast_shape)
        True (3, 3, 3)
    """
    max_dim = max(A.dim(), B.dim())
    # Adjust dimensions of A and B to match the higher dimensionality
    while A.dim() < max_dim:
        A = A.unsqueeze(0)  # Prepend dimensions to A
    while B.dim() < max_dim:
        B = B.unsqueeze(0)  # Prepend dimensions to B

    boradcast_shape = []

    for i,j in zip(A.shape, B.shape):
      if i == j or i == 1 or j == 1:
          boradcast_shape.append(max(i,j))
      else:
          return (False, None)

    return (True, tuple(boradcast_shape))

# Usage example
A = torch.rand(3, 3, 3)
B = torch.arange(9).reshape(3, 3)
print(broad_together(B, A))

(True, (3, 3, 3))


### סעיף ג

In [4]:
def new_broadcast_tensors(A: Tensor, B: Tensor) -> (Tensor, Tensor):
    """
    Broadcasts two input tensors, A and B, to a common shape if they are compatible for broadcasting,
    according to the PyTorch broadcasting rules. This function first checks if the tensors can be
    broadcast together using the broad_together function. If they can, it uses the new_expand_as
    function to broadcast each tensor to the common shape.

    Args:
        A (Tensor): The first tensor to be broadcast.
        B (Tensor): The second tensor to be broadcast.

    Returns:
        (Tensor, Tensor): A tuple containing two tensors broadcast to the common shape.

    Raises:
        ValueError: If tensors A and B cannot be broadcast together according to broadcasting rules.

    Example:
        >>> A = torch.tensor([1, 2, 3])
        >>> B = torch.tensor([[0], [1], [2]])
        >>> C, D = new_broadcast_tensors(A, B)
        >>> print(C)
        >>> print(D)
        tensor([[1, 2, 3],
                [1, 2, 3],
                [1, 2, 3]])
        tensor([[0, 0, 0],
                [1, 1, 1],
                [2, 2, 2]])
    """
    can_broadcast, broadcast_shape = broad_together(A, B)
    if not can_broadcast:
          raise RuntimeError(f"expand_as: The size of shape A: ({A.shape}) must match the shape of tensor B ({ B.shape}) at non-singleton dimension.")

    # Broadcasting A and B to the common shape.
    C = new_expand_as(A, torch.empty(broadcast_shape))
    D = new_expand_as(B, torch.empty(broadcast_shape))

    return C, D

# Usage example
x = torch.arange(3).reshape(1,3)
y = torch.arange(2).reshape(2,1)
print(x, y, sep='\n\n')
print('\n')
a,b = torch.broadcast_tensors(x, y)
print(a, b, sep='\n\n')

tensor([[0, 1, 2]])

tensor([[0],
        [1]])


tensor([[0, 1, 2],
        [0, 1, 2]])

tensor([[0, 0, 0],
        [1, 1, 1]])


### סעיף ד

expand_as function

In [5]:
x = torch.rand(3, 2, 1)
y = torch.rand(2,1)

print(f'Built-in PyTorch expand_as: {y.expand_as(x)}')
print('\n')
print(f'Custom Function new_expand_as: {new_expand_as(y,x)}')

Built-in PyTorch expand_as: tensor([[[0.0605],
         [0.1956]],

        [[0.0605],
         [0.1956]],

        [[0.0605],
         [0.1956]]])


Custom Function new_expand_as: tensor([[[0.0605],
         [0.1956]],

        [[0.0605],
         [0.1956]],

        [[0.0605],
         [0.1956]]])


In [6]:
x = torch.rand(1, 1)
y = torch.rand(2,1)

try:
    result = y.expand_as(x)
    print(f'Built-in PyTorch expand_as result: {result}')
except RuntimeError as e:
    print(f'Built-in PyTorch expand_as: RuntimeError - {e}')

print('\n')  # Newline for readability

# Attempt to use your custom new_expand_as function
try:
    result = new_expand_as(y, x)
    print(f'Custom Function new_expand_as result: {result}')
except RuntimeError as e:
    print(f'Custom Function new_expand_as: RuntimeError - {e}')


Built-in PyTorch expand_as: RuntimeError - The expanded size of the tensor (1) must match the existing size (2) at non-singleton dimension 0.  Target sizes: [1, 1].  Tensor sizes: [2, 1]


Custom Function new_expand_as: RuntimeError - expand_as: The size of the source tensor (torch.Size([2, 1])) must match the size of the target tensor (torch.Size([1, 1])) at non-singleton dimension.


In [7]:
x = torch.rand(3, 2, 1,5,6)
y = torch.rand(5,6)

print(f'Built-in PyTorch expand_as: {y.expand_as(x)}')
print('\n')
print(f'Custom Function new_expand_as: {new_expand_as(y,x)}')

Built-in PyTorch expand_as: tensor([[[[[0.3339, 0.3446, 0.5054, 0.7979, 0.4828, 0.2407],
           [0.0280, 0.3453, 0.8613, 0.2380, 0.1369, 0.0485],
           [0.7146, 0.2735, 0.3231, 0.5393, 0.8435, 0.5303],
           [0.4798, 0.6109, 0.9414, 0.5679, 0.2877, 0.4652],
           [0.5059, 0.9895, 0.9409, 0.3458, 0.6983, 0.1570]]],


         [[[0.3339, 0.3446, 0.5054, 0.7979, 0.4828, 0.2407],
           [0.0280, 0.3453, 0.8613, 0.2380, 0.1369, 0.0485],
           [0.7146, 0.2735, 0.3231, 0.5393, 0.8435, 0.5303],
           [0.4798, 0.6109, 0.9414, 0.5679, 0.2877, 0.4652],
           [0.5059, 0.9895, 0.9409, 0.3458, 0.6983, 0.1570]]]],



        [[[[0.3339, 0.3446, 0.5054, 0.7979, 0.4828, 0.2407],
           [0.0280, 0.3453, 0.8613, 0.2380, 0.1369, 0.0485],
           [0.7146, 0.2735, 0.3231, 0.5393, 0.8435, 0.5303],
           [0.4798, 0.6109, 0.9414, 0.5679, 0.2877, 0.4652],
           [0.5059, 0.9895, 0.9409, 0.3458, 0.6983, 0.1570]]],


         [[[0.3339, 0.3446, 0.5054, 0.7979,

In [9]:
x = torch.rand(2,2)
y = torch.rand(1,1,1,1)

try:
    result = y.expand_as(x)
    print(f'Built-in PyTorch expand_as result: {result}')
except RuntimeError as e:
    print(f'Built-in PyTorch expand_as: RuntimeError - {e}')

print('\n')  # Newline for readability

# Attempt to use your custom new_expand_as function
try:
    result = new_expand_as(y, x)
    print(f'Custom Function new_expand_as result: {result}')
except RuntimeError as e:
    print(f'Custom Function new_expand_as: RuntimeError - {e}')


Built-in PyTorch expand_as: RuntimeError - expand(torch.FloatTensor{[1, 1, 1, 1]}, size=[2, 2]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (4)


Custom Function new_expand_as: RuntimeError - expand_as: The source tensor cannot have more dimensions than the target tensor.


In [10]:
x = torch.rand(2,2,5,6,1,1)
y = torch.rand(5,6,1,1)

print(f'Built-in PyTorch expand_as: {y.expand_as(x)}')
print('\n')
print(f'Custom Function new_expand_as: {new_expand_as(y,x)}')

Built-in PyTorch expand_as: tensor([[[[[[0.6685]],

           [[0.4454]],

           [[0.1453]],

           [[0.6560]],

           [[0.4037]],

           [[0.9545]]],


          [[[0.9027]],

           [[0.5350]],

           [[0.7284]],

           [[0.3735]],

           [[0.8037]],

           [[0.3534]]],


          [[[0.5313]],

           [[0.6079]],

           [[0.4667]],

           [[0.2500]],

           [[0.5297]],

           [[0.3543]]],


          [[[0.2114]],

           [[0.3548]],

           [[0.9892]],

           [[0.3860]],

           [[0.8929]],

           [[0.5092]]],


          [[[0.1782]],

           [[0.2417]],

           [[0.4081]],

           [[0.1372]],

           [[0.9968]],

           [[0.1361]]]],



         [[[[0.6685]],

           [[0.4454]],

           [[0.1453]],

           [[0.6560]],

           [[0.4037]],

           [[0.9545]]],


          [[[0.9027]],

           [[0.5350]],

           [[0.7284]],

           [[0.3735]],

torch.broadcast_tensors

In [11]:
x = torch.rand(3, 2, 1)
y = torch.rand(2,1)

print(f'Built-in PyTorch expand_as: {torch.broadcast_tensors(x,y)}')
print('\n')
print(f'Custom Function new_expand_as: {new_broadcast_tensors(x,y)}')

Built-in PyTorch expand_as: (tensor([[[0.0450],
         [0.1396]],

        [[0.4657],
         [0.0481]],

        [[0.3457],
         [0.6171]]]), tensor([[[0.6849],
         [0.0965]],

        [[0.6849],
         [0.0965]],

        [[0.6849],
         [0.0965]]]))


Custom Function new_expand_as: (tensor([[[0.0450],
         [0.1396]],

        [[0.4657],
         [0.0481]],

        [[0.3457],
         [0.6171]]]), tensor([[[0.6849],
         [0.0965]],

        [[0.6849],
         [0.0965]],

        [[0.6849],
         [0.0965]]]))


In [12]:
x = torch.rand(3, 1, 2)
y = torch.rand(2)

print(f'Built-in PyTorch expand_as: {torch.broadcast_tensors(x,y)}')
print('\n')
print(f'Custom Function new_expand_as: {new_broadcast_tensors(x,y)}')

Built-in PyTorch expand_as: (tensor([[[0.8606, 0.4244]],

        [[0.3882, 0.1140]],

        [[0.2720, 0.7793]]]), tensor([[[0.3793, 0.6160]],

        [[0.3793, 0.6160]],

        [[0.3793, 0.6160]]]))


Custom Function new_expand_as: (tensor([[[0.8606, 0.4244]],

        [[0.3882, 0.1140]],

        [[0.2720, 0.7793]]]), tensor([[[0.3793, 0.6160]],

        [[0.3793, 0.6160]],

        [[0.3793, 0.6160]]]))


In [13]:
x = torch.rand(3,1,1)
y = torch.rand(1,2,3)

print(f'Built-in PyTorch expand_as: {torch.broadcast_tensors(x,y)}')
print('\n')
print(f'Custom Function new_expand_as: {new_broadcast_tensors(x,y)}')

Built-in PyTorch expand_as: (tensor([[[0.2342, 0.2342, 0.2342],
         [0.2342, 0.2342, 0.2342]],

        [[0.2804, 0.2804, 0.2804],
         [0.2804, 0.2804, 0.2804]],

        [[0.4202, 0.4202, 0.4202],
         [0.4202, 0.4202, 0.4202]]]), tensor([[[0.2078, 0.2814, 0.5411],
         [0.5726, 0.6154, 0.3828]],

        [[0.2078, 0.2814, 0.5411],
         [0.5726, 0.6154, 0.3828]],

        [[0.2078, 0.2814, 0.5411],
         [0.5726, 0.6154, 0.3828]]]))


Custom Function new_expand_as: (tensor([[[0.2342, 0.2342, 0.2342],
         [0.2342, 0.2342, 0.2342]],

        [[0.2804, 0.2804, 0.2804],
         [0.2804, 0.2804, 0.2804]],

        [[0.4202, 0.4202, 0.4202],
         [0.4202, 0.4202, 0.4202]]]), tensor([[[0.2078, 0.2814, 0.5411],
         [0.5726, 0.6154, 0.3828]],

        [[0.2078, 0.2814, 0.5411],
         [0.5726, 0.6154, 0.3828]],

        [[0.2078, 0.2814, 0.5411],
         [0.5726, 0.6154, 0.3828]]]))


In [14]:
x = torch.rand(1,2,1)
y = torch.rand(2,1)

print(f'Built-in PyTorch expand_as: {torch.broadcast_tensors(x,y)}')
print('\n')
print(f'Custom Function new_expand_as: {new_broadcast_tensors(x,y)}')

Built-in PyTorch expand_as: (tensor([[[0.9171],
         [0.1911]]]), tensor([[[0.6841],
         [0.1534]]]))


Custom Function new_expand_as: (tensor([[[0.9171],
         [0.1911]]]), tensor([[[0.6841],
         [0.1534]]]))


In [15]:
x = torch.rand(1,3,2,4)
y = torch.rand(1,2,3,4)

try:
    result = torch.broadcast_tensors(x,y)
    print(f'Built-in PyTorch expand_as result: {result}')
except RuntimeError as e:
    print(f'Built-in PyTorch expand_as: RuntimeError - {e}')

print('\n')  # Newline for readability

# Attempt to use your custom new_expand_as function
try:
    result = new_broadcast_tensors(x,y)
    print(f'Custom Function new_expand_as result: {result}')
except RuntimeError as e:
    print(f'Custom Function new_expand_as: RuntimeError - {e}')


Built-in PyTorch expand_as: RuntimeError - The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 2


Custom Function new_expand_as: RuntimeError - expand_as: The size of shape A: (torch.Size([1, 3, 2, 4])) must match the shape of tensor B (torch.Size([1, 2, 3, 4])) at non-singleton dimension.


In [16]:
x = torch.rand(1,3,2,4,4,4)
y = torch.rand(1,3,2,1,4,1)

try:
    result = torch.broadcast_tensors(x,y)
    print(f'Built-in PyTorch expand_as result: {result}')
except RuntimeError as e:
    print(f'Built-in PyTorch expand_as: RuntimeError - {e}')

print('\n')  # Newline for readability

# Attempt to use your custom new_expand_as function
try:
    result = new_broadcast_tensors(x,y)
    print(f'Custom Function new_expand_as result: {result}')
except RuntimeError as e:
    print(f'Custom Function new_expand_as: RuntimeError - {e}')


Built-in PyTorch expand_as result: (tensor([[[[[[5.9489e-01, 3.7390e-01, 6.5592e-01, 1.6016e-01],
            [8.5774e-01, 2.3958e-01, 3.8752e-01, 5.2189e-01],
            [8.4359e-02, 3.3185e-01, 9.7628e-01, 9.6863e-01],
            [1.9198e-01, 7.9117e-02, 4.5457e-01, 3.1727e-01]],

           [[1.4621e-01, 2.1859e-01, 6.7634e-02, 5.6933e-01],
            [1.5976e-01, 2.2818e-01, 4.3407e-01, 2.5148e-01],
            [2.1769e-01, 4.7259e-01, 4.7289e-01, 3.6035e-01],
            [3.1312e-01, 8.3445e-01, 8.2336e-01, 5.4761e-01]],

           [[2.4839e-01, 7.4842e-01, 9.0397e-01, 3.6518e-01],
            [7.8055e-01, 1.5179e-01, 3.1445e-02, 5.4399e-01],
            [3.2665e-01, 9.3265e-01, 5.8034e-01, 4.5060e-01],
            [5.6894e-01, 2.6298e-02, 7.8849e-01, 9.6303e-01]],

           [[6.2809e-01, 1.6547e-01, 4.8911e-01, 2.7286e-01],
            [9.0458e-01, 7.3731e-01, 7.3335e-01, 8.7035e-01],
            [6.5777e-01, 5.6197e-01, 7.4077e-01, 9.2477e-01],
            [7.1126e-01, 3.9